In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
import optuna

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data import**

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv", low_memory=False)#, nrows=10000)
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv", low_memory=False)#, nrows=10000)
train.info(memory_usage="deep")

In [ ]:
test.info(memory_usage="deep")

# **EDA**

In [ ]:
# Colors to be used for plots
colors = ["lightcoral", "sandybrown", "darkorange", "mediumseagreen",
          "lightseagreen", "cornflowerblue", "mediumpurple", "palevioletred",
          "lightskyblue", "sandybrown", "yellowgreen", "indianred",
          "lightsteelblue", "mediumorchid", "deepskyblue"]

In [ ]:
train.head()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6))
pie = ax.pie([len(train), len(test)],
             labels=["Train dataset", "Test dataset"],
             colors=["salmon", "teal"],
             textprops={"fontsize": 15},
             autopct='%1.1f%%')
ax.axis("equal")
ax.set_title("Dataset length comparison", fontsize=18)
fig.set_facecolor('white')
plt.show();

In [ ]:
train.describe()

In [ ]:
features = [x for x in train.columns if x[0]=="f"]

df = pd.concat([train[features], test[features]], axis=0)
df.reset_index(inplace=True, drop=True)

unique_values = df[features].nunique() < 30
cat_features = unique_values[unique_values==True].index
unique_values = df[features].nunique() >= 30
num_features = unique_values[unique_values==True].index

print(f"There are {len(cat_features)} categorical features: {cat_features}")
print(f"There are {len(num_features)} continuous features: {num_features}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

bars = ax.bar(train["target"].value_counts().index,
              train["target"].value_counts().values,
              color=colors,
              edgecolor="black",
              width=0.4)
ax.set_title("Target values distribution", fontsize=20, pad=15)
ax.set_ylabel("Amount of values", fontsize=14, labelpad=15)
ax.set_xlabel("Target value", fontsize=14, labelpad=10)
ax.set_xticks(train["target"].value_counts().index)
ax.tick_params(axis="both", labelsize=14)
ax.bar_label(bars, [f"{x:2.2f}%" for x in train["target"].value_counts().values/(len(train)/100)],
                 padding=5, fontsize=15)
ax.bar_label(bars, [f"{x:2d}" for x in train["target"].value_counts().values],
                 padding=-30, fontsize=15)
ax.margins(0.2, 0.12)
ax.grid(axis="y")

plt.show();

The target value classes are balanced which is good.

In [ ]:
train.isna().sum().sum(), test.isna().sum().sum()

There are no missing values in the both datasets.

Let's check feature values distribution in the both datasets.

In [ ]:
df = pd.concat([train[num_features], test[num_features]], axis=0)
columns = df.columns.values

cols = 5
rows = len(columns) // cols + 1

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,65), sharex=False)

plt.subplots_adjust(hspace = 0.3)
i=0

for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns):
            axs[r, c].set_visible(False)
        else:
            hist1 = axs[r, c].hist(train[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="deepskyblue",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train Dataset")
            hist2 = axs[r, c].hist(test[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="palevioletred",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Test Dataset")
            axs[r, c].set_title(columns[i], fontsize=12, pad=5)
            axs[r, c].set_yticks(axs[r, c].get_yticks())
            axs[r, c].set_yticklabels([str(int(i/1000))+"k" for i in axs[r, c].get_yticks()])
            axs[r, c].tick_params(axis="y", labelsize=10)
            axs[r, c].tick_params(axis="x", labelsize=10)
            axs[r, c].grid(axis="y")
            if i == 0:
                axs[r, c].legend(fontsize=10)
                                  
        i+=1
#plt.suptitle("Numerical feature values distribution in both datasets", y=0.99)
plt.show();

Some features look like categorical ones. Maybe it's worth to try to transform them into bins.

As you can see, the datasets are well balanced. So target distribution should probably be the same for test predictions.

In [ ]:
print("Numerical features with the least amount of unique values:")
train[num_features].nunique().sort_values().head(5)

Let's look at feature correlation.

In [ ]:
# Plot dataframe
df = train[features].corr().round(5)

# Mask to hide upper-right part of plot as it is a duplicate
mask = np.zeros_like(df)
mask[np.triu_indices_from(mask)] = True

# Making a plot
plt.figure(figsize=(16,16))
ax = sns.heatmap(df, annot=False, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})
ax.set_title("Feature correlation heatmap", fontsize=17)
plt.setp(ax.get_xticklabels(), rotation=90, ha="right",
         rotation_mode="anchor", weight="normal")
plt.setp(ax.get_yticklabels(), weight="normal",
         rotation_mode="anchor", rotation=0, ha="right")
plt.show();

There is very weak linear correlation between the features.

# **Data preprocessing**

In [ ]:
# Scaling all values
s_scaler = StandardScaler()
for col in features:
    train[col] = s_scaler.fit_transform(np.array(train[col]).reshape(-1,1))
    test[col] = s_scaler.transform(np.array(test[col]).reshape(-1,1))

In [ ]:
X = train[features].copy()
X_test = test[features].copy()
y = train["target"]

# **Hyperparameters optimization**

Hyperparameters used in this notebook were optimized using Optuna. The code used or that is shown below. They are commented in order to save runtime as optimization has been already done.

In [ ]:
# def train_model_optuna(trial, X_train, X_valid, y_train, y_valid):
#     """
#     A function to train a model using different hyperparamerters combinations provided by Optuna. 
#     Loss of validation data predictions is returned to estimate hyperparameters effectiveness.
#     """
    
        
#     #A set of hyperparameters to optimize by optuna
#     lgbm_params = {
#                     "objective": trial.suggest_categorical("objective", ['binary']),
#                     "boosting_type": trial.suggest_categorical("boosting_type", ['gbdt']),
#                     "num_leaves": trial.suggest_int("num_leaves", 2, 256),
#                     "max_depth": trial.suggest_int("max_depth", 1, 6),
# #                     "max_depth": trial.suggest_categorical("max_depth", [8]),
#                     "learning_rate": trial.suggest_float("learning_rate", 0.15, 1, step=0.01),
#                     "n_estimators": trial.suggest_categorical("n_estimators", [40000]),        
#                     "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 100.0, step=0.1),
#                     "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 100.0, step=0.1),
#                     "random_state": trial.suggest_categorical("random_state", [42]),
#                     "bagging_seed": trial.suggest_categorical("bagging_seed", [42]),
#                     "feature_fraction_seed": trial.suggest_categorical("feature_fraction_seed", [42]), 
#                     "n_jobs": trial.suggest_categorical("n_jobs", [4]), 
#                     "subsample": trial.suggest_float("subsample", 0.1, 1, step=0.01),
#                     "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
#                     "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1, step=0.01),
# #                     "colsample_bytree": trial.suggest_categorical("colsample_bytree", [1]),
# #                     "device_type": trial.suggest_categorical("device_type", ["GPU"]),
#                     'min_child_samples': trial.suggest_int('min_child_samples', 5, 300),
#                     'min_child_weight': trial.suggest_int('min_child_weight', 256, 512),
        
#                     }



#     # Model loading and training
#     model = LGBMClassifier(**lgbm_params)
#     model.fit(X_train, y_train,
#               eval_set=[(X_valid, y_valid)],
#               eval_metric="auc",
#               early_stopping_rounds=100,
#               verbose=False)
    
#     print(f"Number of boosting rounds: {model.best_iteration_}")
#     oof = model.predict_proba(X_valid)[:, 1]
    
#     return roc_auc_score(y_valid, oof)

In [ ]:
# %%time
# # Splitting data into train and valid folds using target bins for stratification
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Setting optuna verbosity to show only warning messages
# # If the line is uncommeted each iteration results will be shown
# # optuna.logging.set_verbosity(optuna.logging.WARNING)

# time_limit = 3600 * 4

# study = optuna.create_study(direction='maximize')
# study.optimize(lambda trial: train_model_optuna(trial, X_train, X_valid,
#                                                     y_train, y_valid),
# #                n_trials = 2
#                timeout=time_limit
#               )

# # Showing optimization results
# print('Number of finished trials:', len(study.trials))
# print('Best trial parameters:', study.best_trial.params)
# print('Best score:', study.best_value)

# **Model training**

The LinearSVC model will be used first. Its predictions are used as a feature for LightGBM model.

In [ ]:
# Fold splitting parameters
splits = 10
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

# Two zero-filled arrays for out-of-fold and test predictions
linear_oof_preds = np.zeros((X.shape[0],))
linear_test_preds = np.zeros((X_test.shape[0],))
total_mean_auc = 0

# Generating folds and making training and prediction for each of them
for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

    linear_model = LinearSVC(tol=1e-7, penalty='l2', dual=False, max_iter=2000, random_state=42)
    linear_model.fit(X_train, y_train)
    
    # Getting validation data predictions. Each fold model makes predictions on an unseen data.
    # So in the end it will be completely filled with unseen data predictions.
    # It will be used to evaluate hyperparameters performance only.    
    linear_oof_preds[valid_idx] = linear_model.decision_function(X_valid)
    
    # Getting mean test data predictions (i.e. devided by number of splits)
    linear_test_preds += linear_model.decision_function(X_test) / splits
    
    # Getting score for a fold model
    fold_auc = roc_auc_score(y_valid, linear_oof_preds[valid_idx])
    print(f"Fold {num} ROC AUC: {fold_auc}")
    
    # Getting mean score of all fold models (i.e. devided by number of splits)
    total_mean_auc += fold_auc / splits

print(f"\nOverall ROC AUC: {total_mean_auc}")

In [ ]:
X["linear_preds"] = linear_oof_preds
X_test["linear_preds"] = linear_test_preds

In [ ]:
# Model hyperparameters
lgbm_params = {'objective': 'binary',
               'boosting_type': 'gbdt',
               'num_leaves': 232,
               'max_depth': 1,
               'learning_rate': 0.18,
               'n_estimators': 40000,
               'reg_alpha': 5.9,
               'reg_lambda': 82.9,
               'random_state': 42,
               'bagging_seed': 42, 
               'feature_fraction_seed': 42,
               'n_jobs': 4,
               'subsample': 0.77, 
               'subsample_freq': 5, 
               'colsample_bytree': 0.91,
               'min_child_samples': 131,
               'min_child_weight': 270}

In [ ]:
%%time
# Setting up fold parameters
splits = 5
skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

# Creating an array of zeros for storing "out of fold" predictions
oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
total_mean_auc = 0

# Generating folds and making training and prediction for the each fold
for num, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.loc[train_idx], X.loc[valid_idx]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    
    model = LGBMClassifier(**lgbm_params)
    model.fit(X_train, y_train,
              verbose=False,
              # These three parameters will stop training before a model starts overfitting 
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric="auc",
              early_stopping_rounds=300,
              )
    
    # Getting mean test data predictions (i.e. devided by number of splits)
    preds += model.predict_proba(X_test)[:, 1] / splits
    
    # Getting mean feature importances (i.e. devided by number of splits)
    model_fi += model.feature_importances_ / splits
    
    # Getting validation data predictions. Each fold model makes predictions on an unseen data.
    # So in the end it will be completely filled with unseen data predictions.
    # It will be used to evaluate hyperparameters performance only.
    oof_preds[valid_idx] = model.predict_proba(X_valid)[:, 1]
    
    # Getting score for a fold model
    fold_auc = roc_auc_score(y_valid, oof_preds[valid_idx])
    print(f"Fold {num} ROC AUC: {fold_auc}")

    # Getting mean score of all fold models (i.e. devided by number of splits)
    total_mean_auc += fold_auc / splits
    
print(f"\nOverall ROC AUC: {total_mean_auc}")

# **Feature importances**

In [ ]:
# Creating a dataframe to be used for plotting
df = pd.DataFrame()
df["Feature"] = X.columns
# Extracting feature importances from the trained model
df["Importance"] = model_fi / model_fi.sum()
# Sorting the dataframe by feature importance
df.sort_values("Importance", axis=0, ascending=False, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 35))
bars = ax.barh(df["Feature"], df["Importance"], height=0.4,
               color="mediumorchid", edgecolor="black")
ax.set_title("Feature importances", fontsize=30, pad=15)
ax.set_ylabel("Feature name", fontsize=20, labelpad=15)
ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=13)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")
# Adding labels on top
ax2 = ax.secondary_xaxis('top')
ax2.set_xlabel("Feature importance", fontsize=20, labelpad=13)
ax2.tick_params(axis="x", labelsize=15)
ax.margins(0.05, 0.01)

# Inverting y axis direction so the values are decreasing
plt.gca().invert_yaxis()

# **Predictions submission**

In [ ]:
predictions = pd.DataFrame()
predictions["id"] = test["id"]
predictions["target"] = preds

predictions.to_csv('submission.csv', index=False, header=predictions.columns)
predictions.head()